In [ ]:
from __future__ import division
import pickle
import os

import numpy as np
from scipy.stats import binom_test, ttest_1samp

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
import matplotlib as mpl
mpl.rc('savefig', dpi=300)
mpl.rc('text', usetex=True)

In [ ]:
data_dir = os.path.join('data', 'quadrotor')

In [ ]:
pilot_ids = ['spike', 'jet', 'faye', 'vicious']

In [ ]:
def onehot_decode(x):
  l = np.nonzero(x)[0]
  assert len(l) == 1
  return l[0]

In [ ]:
NOOP = 8
NOOPS = list(set(range(9)) - {6, 7})
def compute_in_events_for_comb_traj(traj):
  ts = [t for t, obs in enumerate(traj) if onehot_decode(obs[-6:]) not in NOOPS]
  return list(np.array(ts) / len(traj))  

In [ ]:
eval_rollouts = []
comb_rollouts = []
for pilot_id in pilot_ids:
  with open(os.path.join(data_dir, '%s_pilot_eval.pkl' % pilot_id), 'rb') as f:
    eval_rollouts.extend(list(zip(*(pickle.load(f)[pilot_id]))))
  with open(os.path.join(data_dir, '%s_copilot_eval.pkl' % pilot_id), 'rb') as f:
    comb_rollouts.extend(list(zip(*(pickle.load(f)[pilot_id]))))

In [ ]:
def n_act_changes(seq):
  return np.mean([1 if x != y else 0 for x, y in zip(seq[:-1], seq[1:])])

In [ ]:
n_changes = []
rews = []
outcomes = []
ins = []
for rollout in eval_rollouts:
  n_changes.append(n_act_changes(rollout[3]))
  rews.append(rollout[0])
  outcomes.append(rollout[1])
  ts = [t for t, a in enumerate(rollout[3]) if a != NOOP]
  ins.extend(list(np.array(ts) / len(rollout[3])))

In [ ]:
comb_n_changes = []
comb_rews = []
comb_outcomes = []
comb_ins = []
for rollout in comb_rollouts:
  comb_n_changes.append(n_act_changes(rollout[3]))
  comb_rews.append(rollout[0])
  comb_outcomes.append(rollout[1])
  ts = [t for t, a in enumerate(rollout[3]) if a not in NOOPS]
  comb_ins.extend(list(np.array(ts) / len(rollout[3])))

In [ ]:
outcomes = [x if x in [-100, 100] else 0 for x in outcomes]
comb_outcomes = [x if x in [-100, 100] else 0 for x in comb_outcomes]

In [ ]:
mpl.rcParams.update({'font.size': 16})

In [ ]:
plt.xlabel('Fraction of Actions that Differ from Previous Action')
plt.ylabel('Reward')
plt.scatter(n_changes, rews, label='Solo Human Pilot', alpha=0.25, color='gray')
plt.scatter(comb_n_changes, comb_rews, label='Human Pilot + Copilot', alpha=0.25, color='orange')
plt.legend(loc='best')
plt.show()

In [ ]:
all_n_changes = np.array(n_changes + comb_n_changes)
all_outcomes = np.array(outcomes + comb_outcomes)
n_changes = np.array(n_changes)
comb_n_changes = np.array(comb_n_changes)
outcomes = np.array(outcomes)
comb_outcomes = np.array(comb_outcomes)

In [ ]:
plt.xlabel('Fraction of Actions that Differ from Previous Action')
plt.ylabel('Frequency (Normalized Number of Trajectories)')
plt.title('Assisted Trajectories')
plt.hist(comb_n_changes[comb_outcomes == 100], alpha=0.5, linewidth=0, label='Success', color='green', normed=True)
plt.hist(comb_n_changes[comb_outcomes == -100], alpha=0.5, linewidth=0, label='Crash', color='red', normed=True)
plt.legend(loc='best')
plt.show()

In [ ]:
plt.xlabel('Fraction of Actions that Differ from Previous Action')
plt.ylabel('Frequency (Normalized Number of Trajectories)')
plt.title('All Trajectories')
plt.hist(all_n_changes[all_outcomes == 100], alpha=0.5, linewidth=0, label='Success', color='green', normed=True)
plt.hist(all_n_changes[all_outcomes == -100], alpha=0.5, linewidth=0, label='Crash', color='red', normed=True)
plt.legend(loc='best')
plt.show()

In [ ]:
plt.xlabel('Fraction of Actions that Differ from Previous Action')
plt.ylabel('Frequency (Normalized Number of Trajectories)')
plt.title('All Trajectories')
plt.hist(n_changes, alpha=0.5, linewidth=0, label='Solo Human Pilot', color='gray', normed=True)
plt.hist(comb_n_changes, alpha=0.5, linewidth=0, label='Human Pilot + Copilot', color='orange', normed=True)
plt.legend(loc='best')
plt.show()

In [ ]:
bin_heights, bin_edges = np.histogram(comb_ins, bins=20, normed=True)
bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])
plt.plot(bin_centers, bin_heights, label='Human Pilot + Copilot', color='orange')

bin_heights, bin_edges = np.histogram(ins, bins=20, normed=True)
bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])
plt.plot(bin_centers, bin_heights, label='Solo Human Pilot', color='gray')

plt.title('Quadrotor Perching User Study')
plt.xlabel('Time (Within Episode)')
plt.ylabel('User Input Rate')
plt.legend(loc='best')
plt.savefig(os.path.join(data_dir, 'quad-input-freq-vs-time.pdf'), bbox_inches='tight')
plt.savefig(os.path.join(data_dir, 'quad-input-freq-vs-time.png'), bbox_inches='tight')
plt.show()

In [ ]:
stats_of_pilot = {}

In [ ]:
for pilot_id in pilot_ids:
  with open(os.path.join(data_dir, '%s_pilot_eval.pkl' % pilot_id), 'rb') as f:
    eval_rollouts = list(zip(*(pickle.load(f)[pilot_id])))
  with open(os.path.join(data_dir, '%s_copilot_eval.pkl' % pilot_id), 'rb') as f:
    copi_eval_rollouts = list(zip(*(pickle.load(f)[pilot_id])))
  unassisted_succ = [1 if x[1] == 100 else 0 for x in eval_rollouts]
  assisted_succ = [1 if x[1] == 100 else 0 for x in copi_eval_rollouts]
  unassisted_fail = [1 if x[1] == -100 else 0 for x in eval_rollouts]
  assisted_fail = [1 if x[1] == -100 else 0 for x in copi_eval_rollouts]
  stats_of_pilot[pilot_id] = (
    np.mean(unassisted_succ), np.std(unassisted_succ) / np.sqrt(len(unassisted_succ)), len(unassisted_succ),
    np.mean(assisted_succ), np.std(assisted_succ) / np.sqrt(len(assisted_succ)), len(assisted_succ),
    np.mean(unassisted_fail), np.std(unassisted_fail) / np.sqrt(len(unassisted_fail)), len(unassisted_fail),
    np.mean(assisted_fail), np.std(assisted_fail) / np.sqrt(len(assisted_fail)), len(assisted_fail)
  )

In [ ]:
out = []
for pilot_id in pilot_ids:
  s = stats_of_pilot[pilot_id]
  ctrl_succ = s[0]
  ctrl_fail = s[6]
  treat_succ = s[3]
  treat_fail = s[9]
  out.append([pilot_id, 0, ctrl_succ, ctrl_fail])
  out.append([pilot_id, 1, treat_succ, treat_fail])

In [ ]:
# for JMP
with open(os.path.join(data_dir, 'quad_hyp_test.csv'), 'w') as f:
  f.write('userid,assisted,successrate,crashrate\n')
  f.write('\n'.join([','.join([str(z) for z in x]) for x in out]))

In [ ]:
solo_pi_succs = []
solo_pi_crashes = []
comb_succs = []
comb_crashes = []
for pilot_id in pilot_ids:
  s = stats_of_pilot[pilot_id]
  ctrl_succ = s[0]
  ctrl_fail = s[6]
  treat_succ = s[3]
  treat_fail = s[9]
  solo_pi_succs.append(ctrl_succ)
  solo_pi_crashes.append(ctrl_fail)
  comb_succs.append(treat_succ)
  comb_crashes.append(treat_fail)

In [ ]:
mpl.rcParams.update({'font.size': 16})
plt.xlabel('Crash Rate')
plt.ylabel('Success Rate')
plt.title(r'Quadrotor Perching User Study ($n = %d$)' % len(solo_pi_crashes))
plt.scatter(solo_pi_crashes, solo_pi_succs, label='Solo Human Pilot', color='gray', s=100, marker='o')
plt.scatter(comb_crashes, comb_succs, label='Human Pilot + Copilot', color='orange', s=100, marker='^')
plt.scatter([0.0], [0.25], label='Solo Copilot', color='teal', s=100, marker='s')
plt.legend(loc='upper right')
plt.ylim([-0.05, 1.05])
plt.xlim([-0.05, 1.05])
plt.savefig(os.path.join(data_dir, 'quad-user-study-fig.pdf'), bbox_inches='tight')
plt.show()

In [ ]:
for pilot_id in pilot_ids:
  ctrl_p, ctrl_std, ctrl_n, treat_p, treat_std, treat_n = stats_of_pilot[pilot_id][:6]
  print(
    binom_test(ctrl_p * ctrl_n, ctrl_n, treat_p),
    binom_test(treat_p * treat_n, treat_n, ctrl_p)
  )
  ctrl_p, ctrl_std, ctrl_n, treat_p, treat_std, treat_n = stats_of_pilot[pilot_id][6:]
  print(
    binom_test(ctrl_p * ctrl_n, ctrl_n, treat_p),
    binom_test(treat_p * treat_n, treat_n, ctrl_p)
  )

In [ ]:
unassisted_succ = []
assisted_succ = []
unassisted_fail = []
assisted_fail = []
unassisted_rew = []
assisted_rew = []
for pilot_id in pilot_ids:
  with open(os.path.join(data_dir, '%s_pilot_eval.pkl' % pilot_id), 'rb') as f:
    eval_rollouts = list(zip(*(pickle.load(f)[pilot_id])))
  with open(os.path.join(data_dir, '%s_copilot_eval.pkl' % pilot_id), 'rb') as f:
    copi_eval_rollouts = list(zip(*(pickle.load(f)[pilot_id])))
  unassisted_succ.extend([1 if x[1] == 100 else 0 for x in eval_rollouts])
  assisted_succ.extend([1 if x[1] == 100 else 0 for x in copi_eval_rollouts])
  unassisted_fail.extend([1 if x[1] == -100 else 0 for x in eval_rollouts])
  assisted_fail.extend([1 if x[1] == -100 else 0 for x in copi_eval_rollouts])
  unassisted_rew.extend([x[0] for x in eval_rollouts])
  assisted_rew.extend([x[0] for x in copi_eval_rollouts])
stats_of_pilots = (
  np.mean(unassisted_rew), np.std(unassisted_rew) / np.sqrt(len(unassisted_rew)),
  np.mean(assisted_rew), np.std(assisted_rew) / np.sqrt(len(assisted_rew)),
  np.mean(unassisted_succ), np.std(unassisted_succ) / np.sqrt(len(unassisted_succ)),
  np.mean(assisted_succ), np.std(assisted_succ) / np.sqrt(len(assisted_succ)), 
  np.mean(unassisted_fail), np.std(unassisted_fail) / np.sqrt(len(unassisted_fail)), 
  np.mean(assisted_fail), np.std(assisted_fail) / np.sqrt(len(assisted_fail)), 
)

In [ ]:
print('%f \pm %f & %f & %f & %f \pm %f & %f & %f \\\\' % (
  stats_of_pilots[0], stats_of_pilots[1], stats_of_pilots[4], stats_of_pilots[8],
  stats_of_pilots[2], stats_of_pilots[3], stats_of_pilots[6], stats_of_pilots[10],
))

In [ ]:
solo_outcomes = []
solo_traj = []
assisted_outcomes = []
assisted_traj = []
for pilot_id in pilot_ids:
  with open(os.path.join(data_dir, '%s_pilot_eval.pkl' % pilot_id), 'rb') as f:
    eval_rollouts = pickle.load(f)[pilot_id]
  with open(os.path.join(data_dir, '%s_copilot_eval.pkl' % pilot_id), 'rb') as f:
    copi_eval_rollouts = pickle.load(f)[pilot_id]
  solo_outcomes.extend(eval_rollouts[1])
  solo_traj.extend(eval_rollouts[2])
  assisted_outcomes.extend(copi_eval_rollouts[1])
  assisted_traj.extend(copi_eval_rollouts[2])

In [ ]:
SUCCESS = 100
CRASH = -100

In [ ]:
def plot_trajectories(outcomes, trajectories, title, file_name=None, show_goal=True):
  plt.title(title)

  for outcome, trajectory in zip(outcomes, trajectories):
    x, y, z, w, vx, vy, vz, vw, dx, dy = list(zip(*trajectory[::5]))
    if outcome == SUCCESS:
      cmap = mpl.cm.YlGn
    elif outcome == CRASH:
      cmap = mpl.cm.YlOrRd
    else:
      cmap = mpl.cm.gray
    plt.scatter(x, y, c=range(len(x)), cmap=cmap, alpha=0.75, linewidth=0)
    if show_goal:
      plt.scatter([0], [0], marker='*', color='yellow', edgecolor='black', linewidth=1, s=300, alpha=0.5)
    
  plt.xlim([-2, 2])
  plt.ylim([-2, 2])
  plt.xticks([])
  plt.yticks([])
  plt.axis('off')
  if file_name is not None:
    plt.savefig(os.path.join(data_dir, file_name), bbox_inches='tight')
  plt.show()

In [ ]:
plot_trajectories(solo_outcomes, solo_traj, 'Solo Human Pilot', 'human-pilot-solo-traj.pdf')

In [ ]:
plot_trajectories(assisted_outcomes, assisted_traj, 'Human Pilot + Copilot', 'human-pilot-assisted-traj.pdf')

In [ ]:
survey_of_pilot = {
  'vicious': [7, 7, 6, 7, 7, 7, 6, 7, 7, 3, 5, 3, 5, 5, 7]
  'jet': [1, 5, 6, 7, 7, 7, 7, 1, 7, 7, 7, 1, 4, 7, 1]
  'faye': [1, 5, 3, 7, 6, 7, 6, 4, 7, 6, 5, 4, 3, 7, 6]
  'spike': [5, 4, 6, 6, 6, 7, 3, 4, 7, 4, 3, 1, 2, 6, 7]
}

In [ ]:
survey_questions = [
  'I have flown a quadcopter before',
  'I improved over time',
  'The game was too difficult',
  'The copilot was generally helpful in completing the task',
  'The copilot made the task less stressful',
  'I understood what the copilot was trying to do',
  "I could anticipate the copilot's behavior",
  'The copilot improved over time',
  'The copilot was helpful in landing on the pad',
  'The copilot was helpful in avoiding flying out of bounds',
  'The copilot was helpful in landing quickly before time ran out',
  'The copilot was helpful in pointing the camera',
  "The copilot was too aggressive and didn't give me enough control",
  'I play differently with the copilot than without the copilot',
  'I prefer playing with the copilot'
]

In [ ]:
assert all(len(v) == len(survey_questions) for v in survey_of_pilot.values())

In [ ]:
survey_elts = [[] for _ in range(len(list(survey_of_pilot.values())[0]))]
for k, v in survey_of_pilot.items():
  for j, x in enumerate(v):
    survey_elts[j].append(x)

In [ ]:
survey_ps = [ttest_1samp(x, 4)[1] for x in survey_elts]

In [ ]:
survey_ps

In [ ]:
survey_counts = np.zeros((len(survey_of_pilot), len(list(survey_of_pilot.values())[0]), 7))
for i, (k, v) in enumerate(survey_of_pilot.items()):
  for j, x in enumerate(v):
    survey_counts[i, j, (8-x)-1] += 1

In [ ]:
survey_counts = survey_counts.sum(axis=0)

In [ ]:
survey_counts

In [ ]:
question_avgs = (survey_counts * np.tile(np.arange(1, 8, 1), survey_counts.shape[0]).reshape(survey_counts.shape))
question_avgs = question_avgs.sum(axis=1) / len(pilot_ids)

In [ ]:
question_avgs

In [ ]:
def fmt_pval(p):
  if p < 0.001:
    return '<0.001'
  elif p < 0.01:
    return '<0.01'
  elif p < 0.05:
    return '<0.05'
  else:
    return '>0.05'

In [ ]:
for i, x in enumerate(survey_questions):
  if survey_ps[i] < 0.05 and question_avgs[i] > 4:
    print('& \\textbf{%s} & $\\mathbf{%s}$ & $\\mathbf{%0.2f}$ & %d & %d & %d & %d & %d & %d & %d \\\\' % (x, fmt_pval(survey_ps[i]), round(question_avgs[i], 2), *list(survey_counts[i])))
  else:
    print('& %s & $%s$ & %0.2f & %d & %d & %d & %d & %d & %d & %d \\\\' % (x, fmt_pval(survey_ps[i]), round(question_avgs[i], 2), *list(survey_counts[i])))